# Getting Started with Function Calling

We will be using the [OpenAI APIs](https://platform.openai.com/) for all examples.

https://platform.openai.com/docs/guides/function-calling

---

## 3. Function Calling with LLMs

Objectives
- Understand the concept of function calling with LLMs
- Load the necessary libraries
- Learn how to define and use function calling

Below we are loading the necessary libraries, utilities, and configurations.

In [0]:
# update or install the necessary libraries
!pip install openai
!pip install --upgrade typing_extensions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 964.9/964.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.7/348.7 kB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-ffa56110-b521-42b4-bb4b-a5229aa243af
    Can't uninstall 'typing_extensions'. No files were found to uninstall.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to res

In [0]:
dbutils.library.restartPython()

# The tool calling flow

Tool calling is a multi-step conversation between your application and a model via the OpenAI API. The tool calling flow has five high-level steps:

1. Make a request to the model with tools it could call
2. Receive a tool call from the model
3. Execute code on the application side with input from the tool call
4. Make a second request to the model with the tool output
5. Receive a final response from the model (or more tool calls)

<img src="https://cdn.openai.com/API/docs/images/function-calling-diagram-steps.png" alt="Function Calling Diagram" width="550"/>

In [0]:
from openai import OpenAI
import os
import json
import IPython
from datetime import datetime
from zoneinfo import ZoneInfo


DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = OpenAI(
  api_key=DATABRICKS_TOKEN,
  base_url="https://adb-3750392177977863.3.azuredatabricks.net/serving-endpoints"
)

### 3.1 Example

In [0]:
# Simplified timezone data
TIMEZONE_DATA = {
    "tokyo": "Asia/Tokyo",
    "san francisco": "America/Los_Angeles",
    "paris": "Europe/Paris"
}

def get_current_time(location):
    """Get the current time for a given location"""
    print(f"get_current_time called with location: {location}")  
    location_lower = location.lower()
    
    for key, timezone in TIMEZONE_DATA.items():
        if key in location_lower:
            print(f"Timezone found for {key}")  
            current_time = datetime.now(ZoneInfo(timezone)).strftime("%I:%M %p")
            return json.dumps({
                "location": location,
                "current_time": current_time
            })
    
    print(f"No timezone data found for {location_lower}")  
    return json.dumps({"location": location, "current_time": "unknown"})

In [0]:
get_current_time("paris")

get_current_time called with location: paris
Timezone found for paris


'{"location": "paris", "current_time": "03:46 PM"}'

In [0]:
# Define the function for the model
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get the current time in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city name, e.g. San Francisco",
                    },
                },
                "required": ["location"],
            },
        }
    }
]

In [0]:
messages = [{"role": "user", "content": "What's the current time in Paris"}] # Single function call

response = client.chat.completions.create(
  messages=messages,
  model="databricks-gpt-oss-120b",
  tools=tools,
  tool_choice="auto",
)

In [0]:
response_message = response.choices[0].message
print(response_message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_da37093c-16c6-4d2f-8b39-161b9a8278ee', function=Function(arguments='{\n  "location": "Paris"\n}', name='get_current_time'), type='function')])


In [0]:
messages.append(response_message)

# Handle function calls
if response_message.tool_calls:
    for tool_call in response_message.tool_calls:
        if tool_call.function.name == "get_current_time":
            function_args = json.loads(tool_call.function.arguments)
            print(f"Function arguments: {function_args}")  
            time_response = get_current_time(
                location=function_args.get("location")
            )
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": "get_current_time",
                "content": time_response,
            })
else:
    print("No tool calls were made by the model.")  

Function arguments: {'location': 'Paris'}
get_current_time called with location: Paris
Timezone found for paris


In [0]:
# Second API call: Get the final response from the model

final_response = client.chat.completions.create(
  messages=messages,
  model="databricks-gpt-oss-120b",
  tools=tools,
  tool_choice="auto",
)

In [0]:
print(final_response.choices[0].message.content)

The current time in Paris is 03:55 PM.
